<a href="https://colab.research.google.com/github/sanjprasad/Classification/blob/main/Updated_HW_Classification_Random_Forest%2C_SVM_and_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and preparing the data

In this section, I'm importing the proper libraries that will be used in this assignment. I'm also loading the dataset and formatting it to be used. Finally, I create the testing and training sets that will be used with this classification.

In [12]:
from pandas.core.window.indexers import VariableOffsetWindowIndexer
#importing pandas and numpy the standard libraries
import pandas as pd
import numpy as np

#import standard classification tools from sklearn
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import itertools

# import model class (SVC stands for Support Vector Classification)
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import scale
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#visualization imports
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

template = 'ggplot2'
test_size = 0.8 #increased the test size as some of the pathologic states were missing

#loading in the dataset - I combined the dataset and the pathologic stage variable in excel and uploaded my dataset to colab instead. 
#for the updated hw, I updated the excel to remove the samples with not stage data, and kept only stage I and II. All other stages were categorized in other. 
data = pd.read_table('/content/updatedclassificationHW.csv', sep=',', header=(0))

#prepare data
features = data.iloc[1:, 0].values
data=data.drop(['ID'], axis = 1) 

dataT = np.transpose(data)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
data #making sure the data looks how I want it to

TCGA_DD_A1EC_01A TCGA_ES_A2HT_01A TCGA_DD_A3A8_01A TCGA_DD_A1EL_01A  \
0              Stage I          Stage I         Stage II         Stage II   
1                    0                0                0                0   
2                  9.2                0                2                8   
3                  8.8                3                0                5   
4                  144           104.55           196.73           614.74   
...                ...              ...              ...              ...   
20527          12725.0           5090.0           1877.0           3098.0   
20528           4548.0            963.0            836.0           2076.0   
20529           1494.0            271.0            547.0           1780.0   
20530              8.0              4.0              1.0             14.0   
20531              1.0             33.0            206.0            247.0   

      TCGA_DD_A3A3_01A TCGA_EP_A3RK_01A TCGA_DD_A11D_01A TCGA_DD_A39X_01A  \
0              Stage I            Other          Stage I          Stage I   
1                    0                0                0                0   
2                 3.15             3.31             2.26             3.75   
3                 1.85             8.69             3.74             6.25   
4                54.87           160.63            69.86           225.26   
...                ...              ...              ...              ...   
20527           2584.0           6402.0           4185.0           6012.0   
20528            182.0           1243.0            316.0           1181.0   
20529            284.0            763.0            358.0            343.0   
20530              0.0             10.0              2.0              3.0   
20531              8.0             87.0             74.0            588.0   

      TCGA_DD_A11C_01A TCGA_DD_A11B_01A  ... TCGA_DD_A1EH_01A  \
0              Stage I          Stage I  ...            Other   
1                    0                0  ...                0   
2                11.18            12.91  ...            21.72   
3                10.82             4.09  ...            19.28   
4               167.17           229.61  ...           326.15   
...                ...              ...  ...              ...   
20527           4425.0           4591.0  ...           9132.0   
20528           1175.0           2312.0  ...           1056.0   
20529           1342.0            877.0  ...           1172.0   
20530             11.0              5.0  ...             17.0   
20531            107.0            150.0  ...             28.0   

      TCGA_EP_A12J_01A TCGA_EP_A26S_01A TCGA_FV_A23B_01A TCGA_DD_A39V_01A  \
0              Stage I          Stage I         Stage II         Stage II   
1                    0                0                0                0   
2                14.61            11.42            48.06             3.96   
3                 9.39             1.58            67.94             2.04   
4               180.51           286.78           303.27           126.07   
...                ...              ...              ...              ...   
20527           1616.0           2000.0           6540.0           4873.0   
20528           1680.0           2129.0           1542.0            417.0   
20529           1589.0           1717.0           1768.0            306.0   
20530             18.0             11.0              2.0              4.0   
20531            363.0            247.0              4.0            174.0   

      TCGA_DD_A39W_01A TCGA_DD_A3A2_01A TCGA_FV_A2QR_01A TCGA_DD_A3A4_01A  \
0                Other          Stage I          Stage I            Other   
1                    0                0                0                0   
2                 1.67             4.23                0                0   
3                 3.33             5.77                0                7   
4               216.06            128.6           224.42         

In [14]:
#X is the data, y is the lables (classes)
X = dataT.iloc[:, 1:].values
y = dataT.iloc[:, 0].values

classes = np.unique(y)
n_classes = len(classes)

class_label= dataT[0]

#creating the training and testing sets
X = scale(X, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)



# Applying three classification methods to this data

First looking at some of the sample distribution.



In [15]:
#how many 'Stage I' samples are there in the original dataset?
print(len(y[y == 'Stage I']))

16


In [16]:
#how many 'Stage II' samples are there in the original dataset?
print(len(y[y == 'Stage II']))

8


In [17]:
#how many 'Other' samples are there in the original dataset?
print(len(y[y == 'Other']))

11


In [18]:
#how many '[Not Available]' samples are there in the original dataset? double checking to make sure they were removed
print(len(y[y == '[Not Available]']))

0


In [19]:
#comparing the proportions of the possible stages
for stage in classes:
  print(stage, ": ", round(len(y_test[y_test == stage])/len(y[y == stage]),2))

Other :  0.82
Stage I :  0.75
Stage II :  0.88


Trying the SVC method!

In [20]:
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
#SVC Classification- done just like how it is in the notebook shared with us
# using a linear kernel, it is the only kernel which allows us to use features selection
model = LinearSVC(C=1.0)
model.fit(X_train, y_train)


LinearSVC()

In [21]:
# after training we can predict labels for the test samples and check the accuracy of the model
predictions = model.predict(X_test)

c_matrix = confusion_matrix(y_test,predictions)
cr = classification_report(y_test,predictions, output_dict=True)
model_accuracy = accuracy_score(y_test, predictions)

In [22]:
 model_accuracy

0.2857142857142857

**After updating the data:**
The model accuracy went up from 0.23529411764705882 to 0.2857142857142857, which is around a 0.05 increase 

**Original:**
Above, I trained the model with SVC classification and then predicted the labels for the test samples. I also checked the model's accuracy and found it to be 0.23529411764705882. 

Below I show the classification results!

In [23]:
clsf_report = pd.DataFrame(cr).transpose()
clsf_report1 = clsf_report[0:n_classes]

fig = px.bar(clsf_report1, barmode='group', template=template)
fig.update_layout(width=700, height=700, template=template, title="Classification Report")
fig.show()

Precision is the ratio of true positives to total positives and essentially refers to accuracy, so a higher precision score indicates a low rate of false positives. 

Recall is the ratio of true positives to all the actual observations (so total true positives and false negatives) and essentially refers to sensitivity of the model. 

F1 score is the weighted average of precision and recall so it takes false positives and false negatives into account in the score. F1 score is one of the most important parameter as it can be more useful than precision. 

Support is the number of occurrences of each class.

**After the update:**
The other stage had the highest recall of 0.5, while Stage I had a recall of 0.25. The other stages categorized as other had negligible recall. The F1 score was slightly higher the "Other" category at about 0.357 while the Stage I had a F1 0.285. The precision value was approximetely 0.333 for Stage I which was higher than the "Other" stage of about 0.26. Interestingly, stage II still had super small values for both to the point where they didn't show up on the graph. Additionally, originally stage I had the highest F1 and precision scores. 

**Original:** Looking at the above graph, we see low parameters for all of the classes with the exception of the support parameter. Stage I has the highest recall of 0.5, which is not very good for this model. The F1 scores and precision are all below 0.5 for each class with the highest being 0.387 for F1 score and 0.3157 for precision for stage I. 

In [24]:
#Running PCA

from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA 

dataT1 = dataT
dataT1 = dataT1.iloc[:,1:len(dataT1.columns)]
dataT1

scaled = StandardScaler() 
scaled.fit(dataT1) 
scaled_data = scaled.transform(dataT1) 

n_components=4
pca = PCA(n_components) 
pca.fit(scaled_data) 
xpca = pca.transform(scaled_data)

labels = {str(i): f'PC{i+1}: {pca.explained_variance_ratio_[i]*100:.2f}%' for i in range(n_components)}
template = 'ggplot2'

# x and y given as array_like objects
figPCA2D = px.scatter(xpca, x=0, y=1, color=y, 
                      labels=dataT1.index,
                      hover_name = dataT1.index,
                      title="2D PCA for Labels")
figPCA2D.update_layout(width=1000, height=800, template=template)
figPCA2D.show()

In [25]:
#Creating a confusion matrix of the data
CM_df = pd.DataFrame(c_matrix, columns=classes, index=classes)

heat = go.Heatmap(z = CM_df, x=CM_df.index, 
                  y=CM_df.columns, 
                  showscale = False,
                  text=CM_df,
                  texttemplate="%{text}",
                  textfont={"size":10})
fig = go.Figure(heat)
fig.update_layout(width=700, height=700, template=template, title="Confusion Matrix")
fig.show()

Next, we can do feature selection to get rid of the noise and only see the relevant information. 

In [26]:
# this is only available in the case of a linear kernel
top_features_count = 20

# we use median of weights from all classes
vector_weights = np.median(model.coef_, axis=0)
top_positive_coefficients = np.argsort(vector_weights)[-top_features_count:]
top_negative_coefficients = np.argsort(vector_weights)[:top_features_count]
top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
top_features, top_features_weights = features[top_coefficients], vector_weights[top_coefficients]

d = {'Features':features[top_coefficients], 'Weight':vector_weights[top_coefficients]}
df_features = pd.DataFrame(d)

#plot feature significance
fig = px.bar(y=df_features.Features, x=df_features.Weight, template=template)
fig.update_layout(width=700, height=1000, template=template, title="Feature Significance")
fig.show()

Time to try the Random Forest Classification Method

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#define Random forest model
model = RandomForestClassifier(n_estimators=500, bootstrap=True, verbose=True)

#fit model on training data
model.fit(X_train, y_train)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.5s finished


RandomForestClassifier(n_estimators=500, verbose=True)

In [28]:
#Predict test data and checking the accuracy

predictions = model.predict(X_test)
c_matrix = confusion_matrix(y_test,predictions)

CM_df = pd.DataFrame(c_matrix, columns=classes, index=classes)

cr = classification_report(y_test,predictions, output_dict=True)
model_accuracy = accuracy_score(y_test, predictions)

model_accuracy

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished


0.4642857142857143

**After the update:** We see the model accuracy for the Random Forest classification method is 0.4642857142857143

**Original:**
We see above that the model accuracy for the Random Forest classification method is 0.3235294117647059


In [30]:
clsf_report = pd.DataFrame(cr).transpose()
clsf_report1 = clsf_report[0:n_classes]

fig = px.bar(clsf_report1, barmode='group', template=template)
fig.update_layout(width=700, height=700, template=template, title="Classification Report")
fig.show()

**After the update:** We see in the graph that recall had a 1 for Stage 1, while Stage II had a recall of about 0.14. While precision for stage II has a precision of 1 while Stage I had a precision of 0.444. The F1 score for Stage 1 was 0.615 while stgae II was 0.25. The other category had negiligible values for F1 score, recall and precision. 


**Original:** Above, we see that the recall for Stage 1 is high at almost 0.916666 and with a precision of 0.3928. The recall is good for this class as it's above 0.5, but the precision is still pretty low. The F1 score is also decently good at about 0.55 as well for this class. 

Below we can make a confusion matrix and create a PCA plot:

In [31]:
#Making a confusion matrix
heat = go.Heatmap(z = c_matrix, x=CM_df.index, 
                  y=CM_df.columns, 
                  showscale = False,
                  text=CM_df,
                  texttemplate="%{text}",
                  textfont={"size":10})
fig = go.Figure(heat)
fig.update_layout(width=700, height=700, template=template, title="Confusion Matrix")
fig.show()

In [32]:
dataT1 = dataT
dataT1 = dataT1.iloc[:,1:len(dataT1.columns)]
dataT1

scaled = StandardScaler() 
scaled.fit(dataT1) 
scaled_data = scaled.transform(dataT1) 

#Run Principal Component Analysis
n_components=4
pca = PCA(n_components) 
pca.fit(scaled_data) 
xpca = pca.transform(scaled_data)

labels = {str(i): f'PC{i+1}: {pca.explained_variance_ratio_[i]*100:.2f}%' for i in range(n_components)}
template = 'ggplot2'

# x and y given as array_like objects
figPCA2D = px.scatter(xpca, x=0, y=1, color=y, 
                      labels=dataT1.index,
                      hover_name = dataT1.index,
                      title="2D PCA for Labels")
figPCA2D.update_layout(width=1000, height=800, template=template)
figPCA2D.show()

In [33]:
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
forest_importances = pd.Series(importances, index=features)

RF_df = pd.DataFrame(forest_importances.sort_values(ascending=False))
RF_df.columns = ['Feature']

#plot feature significance
fig = px.bar(RF_df[0:20], template=template)
fig.update_layout(width=1000, height=700, template=template, title="Top 20 Features by Significance")
fig.show()

Above I do feature selection again to remove noise!

In [34]:
!pip uninstall scikit-learn -y -q
!pip install scikit-learn==0.24 -q

     |████████████████████████████████| 22.3 MB 1.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.0 which is incompatible.


Now it's time for the third classification method! Below is Lazy classifier

In [35]:
warnings.filterwarnings('ignore')

!pip install lazypredict -q

In [37]:
#import necessary modules
import time 
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier,LazyRegressor

#Create a lazyClassifier model
model=LazyClassifier(verbose=0,predictions=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=0)

# fit and train the model 
start_time_1=time.time()
models_c,predictions_c=model.fit(X_train, X_test, y_train, y_test)
end_time_1=time.time()

100%|██████████| 29/29 [00:18<00:00,  1.60it/s]


In [39]:
fig = px.bar(models_c[['Time Taken', 'F1 Score','Accuracy']], barmode='group', template=template)
fig.update_layout(width=1200, height=700, template=template, title="Classification Report")
fig.show()

In [40]:
models_c.plot.bar(figsize = (18,8));

In [41]:
models_c.iloc[0:5].plot.bar(figsize = (18,8), title="Top 5 classification models");

# Summary

In this notebook, I used the TCGA Risk Factor dataset and the corresponding clinical annotation data. I utilized Excel in order to organize and reformat the data which included pulling the pathologic state variable and matching it to the corresponding samples in order to have the pathologic state associated with each sample in the dataset with all the genes and expression. This was done by hand as I alphabetized the clinical annotation dataset and then matched the corresponding sample name to add in a row called "Stage" which had the pathologic state. As some of the stages only had one sample, I increased the test size parameter to 0.8 in order to have them all show in the resulting graphs be included in the analysis. 

Following the data preparation, I split the data into test and train datasets, and then I used these sets in my classification models. I began with the Support Vector Classifier (SVC) method, which only had an accuracy of about 23%. This was only further visually shown in the classification report graph, which showed very low scores for all the parameters and the PCA plot which showed no distinct grouping. Stage I seemed to have the most success with this method, although it was still rather low as shown by the parameters in the classification report as we saw the highest recall of 0.5, which is not very good for this model. The F1 scores and precision were 0.387 and 0.3157 respectively for stage I. After this, I used feature selection and found the top 5 marker genes to be FTL2512, APOA2336, SEPP16414, CLU1191 and TF7018 with FTL2512 having the highest value of 0.00208.

The second classification method I tried was the Random Forest Classification Method. This method yielded an accuracy of 32.5%, which was much better than the SVC method. This was shown by the classification report where we can see that the recall for Stage 1 is high at almost 0.916666 and the precision is at 0.3928. The recall is good for this class as it's above 0.5, but the precision is still pretty low. The F1 score is also decently good at about 0.55 as well for this class. Thus, the model showed the most success with Stage I as well. The confusion matrix showed that 10 of the samples were correctly classified with this method. Feature selection was done again with the five marker genes bein ZWINT11130, TAF383860, MRPL965005, PRCC5546, STK2510494, with the corresponding values of 0.0038, 0.0033, 0.0032,0.0030, 0.0027.

Interestingly, the significant features were different for both models, but since neither model had a high accuracy it's hard to determine which is more correct. 

Lastly, I used the Lazy Predict classifier method to look at many of the models that would allow comparing of time taken, F1 scores and accuracy. The top models found were DecisionTreeClassifier, LinearDiscriminantAnalysis, AdaBoostClassifier, KNeighborsClassifer, and SVC. It's interesting that SVC was found to be on this list, considering the accuracy was lower than the Random Forest method. While AdaBoostClassifier was found to be the fastest, it had a low F1 score. The highest F1 score belonged to the Decision Tree Classifier method. As we had discussed in class, F1 scores are a significant parameter to look at and more important than speed, so the Decsion Tree classifier is a model of interest to look at. Interestingly, the accuracy of SVC was fairly high which was different than seen here. 




# Updated Conclusion after feature selection

To do feature selection, I switched how the stages were categorized by keeping stage 1 and stage 2 and making all other stages categorized as "Other". Additionally, I also removed all the samples with no stage data available, which was 7 samples. This was done in Excel and then uploaded into Collab. 

For the Support Vector Classifier method, I saw the model accuracy increase to about 28%, which is a 5% increase from the original model accuracy. From the classification report, we saw that recall was highest for the other category, but precision was higher for stage 1. However, F1 scores were similar for both with the other category just being slightly higher than stage 1. Stage II had negligible recall, precision, and F1 scores as they didn't show up on the graph however. The PCA plot still didn't show distinct grouping of the groups and the confusion matrix indicated that stage II was not classified correctly at all while stage I and the other classification were only partially properly classified. This indicates the model was not able to properly classify the samples. Originally, the top 5 marker genes were found to be FTL2512, APOA2336, SEPP16414, CLU1191 and TF7018 but after the update they were FTL2512, C3718, SERPINA15265, FGA2243 and CLU1191, so only two of the original top 5 five were found again.

Following this, the random forest classification was done and the model accuracy was found to be 46% which was almost a 14% increase from the original model accuracy of 32%. The classification report showed high precision for stage II, much lower recall and F1 scores, and high recall for stage 1, which a decently high precision and a higer F1 score than stage II. This time, the other category had next to negligible results for the parameters. The confusion matrix showed that stage 1 was completely accurately classified, but stage II and other were both very incorrectly classified with the other classification having all samples classified incorrectly. PCA still showed no distinct grouping. Orginally, feature selection was done again with the five marker genes being ZWINT11130, TAF383860, MRPL965005, PRCC5546, STK2510494, but the update gave RHBDD325807, PPID5481, QRFPR84109, PDZK1IP110158 and ARL6IP123204 as the top five marker genes, so none of the original five were found again. 

Finally, from the LazyPredict classifier method, the top 5 models were XGBClassifier, DecisionTreeClassifier, ExtraTreeClassifier, BaggingClassifier and BernoulliNB. Only DecisionTreeClassifier was found to be a top model with the original data. The highest F1 score was found to be XGBClassifer, but the time taken was much higher than the other models. Besides time taken the other parameters are all higher than the other models though, so it would be interesting to further explore. 

Overall, feature selection showed an improvement in accuracy in the models and changed the marker genes and top classification models. This can be further explored in the future by trying other classification methods. 


